In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sklearn.preprocessing import StandardScaler



In [2]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
import psycopg2

In [3]:
# # Path to sqlite
# # Create engine that can talk to the database
database_path = "covid_2.db"
engine = create_engine(f"sqlite:///{database_path}")
# # Make a connection to the SQL database
conn = engine.connect()
# # # Query All Records in the the Database
machine_learning_df = pd.read_sql("SELECT * FROM model_machine_learning_data", conn)
machine_learning_df


,FIPS,State,County Name,Urban Rural Code,Urban Rural Description,Population Estimate 2020,Median Age of County,Total Vaccines,Percent Vaccinated,Case Count,Percent of Population Cases,COVID-19 Deaths,Percent Population Died from Covid,Percent Population Died by Covid above Median Rate,FIPS:1,Adult Obesity Percent,Adult Smoking Percentage,Excessive Drinking Percent
0,6113,CA,Yolo County,2.0,Large fringe metro,219728.0,31.3,132599.0,60.346883,19001.0,8.647510,203.0,0.092387,0.0,6113,0.244,0.137947,0.202009
1,8041,CO,El Paso County,3.0,Medium metro,728310.0,34.6,390344.0,53.595859,83725.0,11.495792,1234.0,0.169433,0.0,8041,0.229,0.152761,0.173437
2,17029,IL,Coles County,5.0,Micropolitan,50383.0,37.0,19345.0,38.395888,7265.0,14.419546,140.0,0.277872,1.0,17029,0.327,0.219284,0.203392
3,13059,GA,Clarke County,4.0,Small metro,127795.0,28.8,29461.0,23.053328,13717.0,10.733597,550.0,0.430377,1.0,13059,0.250,0.185137,0.163662
4,12086,FL,Miami-Dade County,1.0,Large central metro,2707303.0,40.4,2027846.0,74.902809,491834.0,18.166936,7994.0,0.295275,1.0,12086,0.224,0.159370,0.186109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,13215,GA,Muscogee County,3.0,Medium metro,196442.0,34.5,62248.0,31.687725,16312.0,8.303723,926.0,0.471386,1.0,13215,0.379,0.193613,0.175032
795,51540,VA,Charlottesville city,4.0,Small metro,46950.0,32.1,22537.0,48.002130,4472.0,9.525027,204.0,0.434505,1.0,51540,0.300,0.176884,0.180595
796,39155,OH,Trumbull County,3.0,Medium metro,196800.0,44.6,97110.0,49.344512,18896.0,9.601626,441.0,0.224085,0.0,39155,0.392,0.270716,0.170572
797,42005,PA,Armstrong County,2.0,Large fringe metro,64162.0,47.0,34269.0,53.410118,6865.0,10.699479,118.0,0.183909,0.0,42005,0.440,0.233523,0.215703


In [4]:
machine_learning_df = machine_learning_df.filter(["FIPS", "State", "County Name", "Urban Rural Code", "Urban Rural Description", "Population Estimate 2020", "Total Vaccines", "Percent Vaccinated", "Case Count", "Percent of Population Cases", "COVID-19 Deaths", "Median Age of County", "Adult Obesity Percent", "Adult Smoking Percentage","Excessive Drinking Percent", "Percent Population Died from Covid", "Percent Population Died by Covid above Median Rate"])
machine_learning_df.head()


,FIPS,State,County Name,Urban Rural Code,Urban Rural Description,Population Estimate 2020,Total Vaccines,Percent Vaccinated,Case Count,Percent of Population Cases,COVID-19 Deaths,Median Age of County,Adult Obesity Percent,Adult Smoking Percentage,Excessive Drinking Percent,Percent Population Died from Covid,Percent Population Died by Covid above Median Rate
0,6113,CA,Yolo County,2.0,Large fringe metro,219728.0,132599.0,60.346883,19001.0,8.647510,203.0,31.3,0.244,0.137947,0.202009,0.092387,0.0
1,8041,CO,El Paso County,3.0,Medium metro,728310.0,390344.0,53.595859,83725.0,11.495792,1234.0,34.6,0.229,0.152761,0.173437,0.169433,0.0
2,17029,IL,Coles County,5.0,Micropolitan,50383.0,19345.0,38.395888,7265.0,14.419546,140.0,37.0,0.327,0.219284,0.203392,0.277872,1.0
3,13059,GA,Clarke County,4.0,Small metro,127795.0,29461.0,23.053328,13717.0,10.733597,550.0,28.8,0.250,0.185137,0.163662,0.430377,1.0
4,12086,FL,Miami-Dade County,1.0,Large central metro,2707303.0,2027846.0,74.902809,491834.0,18.166936,7994.0,40.4,0.224,0.159370,0.186109,0.295275,1.0


In [5]:
machine_learning_df.to_csv('Final_ML_Data.csv', index=False)

In [6]:
machine_learning_df = machine_learning_df.drop(["State", "County Name", "Urban Rural Description", "COVID-19 Deaths", "Percent Population Died from Covid", "Total Vaccines","Case Count"], axis=1)
machine_learning_df


,FIPS,Urban Rural Code,Population Estimate 2020,Percent Vaccinated,Percent of Population Cases,Median Age of County,Adult Obesity Percent,Adult Smoking Percentage,Excessive Drinking Percent,Percent Population Died by Covid above Median Rate
0,6113,2.0,219728.0,60.346883,8.647510,31.3,0.244,0.137947,0.202009,0.0
1,8041,3.0,728310.0,53.595859,11.495792,34.6,0.229,0.152761,0.173437,0.0
2,17029,5.0,50383.0,38.395888,14.419546,37.0,0.327,0.219284,0.203392,1.0
3,13059,4.0,127795.0,23.053328,10.733597,28.8,0.250,0.185137,0.163662,1.0
4,12086,1.0,2707303.0,74.902809,18.166936,40.4,0.224,0.159370,0.186109,1.0
...,...,...,...,...,...,...,...,...,...,...
794,13215,3.0,196442.0,31.687725,8.303723,34.5,0.379,0.193613,0.175032,1.0
795,51540,4.0,46950.0,48.002130,9.525027,32.1,0.300,0.176884,0.180595,1.0
796,39155,3.0,196800.0,49.344512,9.601626,44.6,0.392,0.270716,0.170572,0.0
797,42005,2.0,64162.0,53.410118,10.699479,47.0,0.440,0.233523,0.215703,0.0


In [7]:
ml_input_df = pd.get_dummies(machine_learning_df, columns= ['FIPS'])

In [8]:
X = ml_input_df.copy()
X = X.drop("Percent Population Died by Covid above Median Rate", axis=1)


 # Create our target
y = ml_input_df["Percent Population Died by Covid above Median Rate"]

In [9]:
X.describe()

,Urban Rural Code,Population Estimate 2020,Percent Vaccinated,Percent of Population Cases,Median Age of County,Adult Obesity Percent,Adult Smoking Percentage,Excessive Drinking Percent,FIPS_1001,FIPS_1003,...,FIPS_55105,FIPS_55117,FIPS_55127,FIPS_55131,FIPS_55133,FIPS_55135,FIPS_55139,FIPS_55141,FIPS_56021,FIPS_56025
count,799.000000,7.990000e+02,799.000000,799.000000,799.000000,799.000000,799.000000,799.000000,799.000000,799.000000,...,799.000000,799.000000,799.000000,799.000000,799.000000,799.000000,799.000000,799.000000,799.000000,799.000000
mean,3.342929,3.105749e+05,49.484180,11.345784,39.816020,0.322205,0.195727,0.189357,0.001252,0.001252,...,0.001252,0.001252,0.001252,0.001252,0.001252,0.001252,0.001252,0.001252,0.001252,0.001252
std,1.321709,5.665322e+05,11.877537,3.769642,4.619412,0.054104,0.039152,0.031906,0.035377,0.035377,...,0.035377,0.035377,0.035377,0.035377,0.035377,0.035377,0.035377,0.035377,0.035377,0.035377
min,1.000000,2.045200e+04,5.157531,0.002340,25.200000,0.143000,0.070756,0.064530,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,7.594600e+04,42.428624,9.214834,37.000000,0.286000,0.169169,0.166427,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,1.519460e+05,49.768712,11.431696,39.600000,0.327000,0.198295,0.187379,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.000000,3.262150e+05,57.316540,14.012820,42.100000,0.361000,0.224197,0.211187,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,6.000000,9.943046e+06,101.577149,20.554329,68.100000,0.503000,0.305477,0.310137,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
# Check the balance of our target values
Counter(y)

Counter({0.0: 398, 1.0: 401})

In [11]:
#Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [12]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
clf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
clf_model = clf_model.fit(X_train_scaled, y_train)

In [14]:
# Calculated the balanced accuracy score
predictions = clf_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, predictions)

0.76

In [15]:
# Display the confusion matrix
confusion_matrix(y_test, predictions)

array([[83, 17],
       [31, 69]], dtype=int64)

In [16]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.73      0.83      0.69      0.78      0.76      0.58       100
        1.0       0.80      0.69      0.83      0.74      0.76      0.56       100

avg / total       0.77      0.76      0.76      0.76      0.76      0.57       200



In [17]:
# List the features sorted in descending order by feature importance
importances = sorted(zip(clf_model.feature_importances_, X.columns), reverse=True)
importances

[(0.11320021067885391, 'Population Estimate 2020'),
 (0.0964529167765377, 'Adult Smoking Percentage'),
 (0.08954873542636192, 'Percent of Population Cases'),
 (0.08430319737238896, 'Excessive Drinking Percent'),
 (0.07612900327444486, 'Percent Vaccinated'),
 (0.07169089865401156, 'Adult Obesity Percent'),
 (0.06898808337308493, 'Median Age of County'),
 (0.045812674864537266, 'Urban Rural Code'),
 (0.0023481287027847034, 'FIPS_19103'),
 (0.0022034196356978853, 'FIPS_36067'),
 (0.0021651864058614217, 'FIPS_27123'),
 (0.0021452251336098282, 'FIPS_51085'),
 (0.0020821317062097826, 'FIPS_47031'),
 (0.0019303408394569306, 'FIPS_40131'),
 (0.001909853143296609, 'FIPS_21125'),
 (0.0018538362536657225, 'FIPS_35025'),
 (0.0018447485252788755, 'FIPS_35043'),
 (0.0017800740095843964, 'FIPS_9003'),
 (0.0017661945636547381, 'FIPS_37021'),
 (0.001743649986100753, 'FIPS_1045'),
 (0.0017339703035226819, 'FIPS_34003'),
 (0.0016927683715570283, 'FIPS_4019'),
 (0.0016747755869857572, 'FIPS_55059'),
 (0.0